In [38]:
import mikeio as mi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gdp
from shapely.geometry import Point, LineString
import sys
from math import atan2, degrees, hypot
import shapely.wkt
import shapely.ops


def reverse_geom(geom):
    def _reverse(x, y, z=None):
        if z:
            return x[::-1], y[::-1], z[::-1]
        return x[::-1], y[::-1]

    return shapely.ops.transform(_reverse, geom)


def get_grid_size(ds):
    # Check if quadratic grid
    if ds.geometry.dx != ds.geometry.dy:
        print('The grid is not quadratic. This might cause insufficient results')
    return ds.geometry.dx


def AngleBtw2Points(pointA, pointB):
    changeInX = pointB[0] - pointA[0]
    changeInY = pointB[1] - pointA[1]
    return degrees(atan2(changeInY, changeInX))


def calculate_flow_direction(perpendicular_angle, col):
    if (col > perpendicular_angle-90) & (col < perpendicular_angle+90):
        return True
    else:
        return False


def calculate_flow_magnitude(df, grid_size, sub_line):
    # Get the columns of the df. The two columns that will be used for further calculations are the P flux (flow in x direction, index = 2) and Q Flux (flow in y direction, index = 3).
    columns = df.columns

    # Resultant flow is given by hypothenuse of both. Note that the flow is given in per meter, meaning that we need to multiply by the grid size
    df['Discharge [meter^3/s]'] = df.apply(
        lambda x: hypot(x[columns[3]], x[columns[2]]), axis=1)
    # df['Discharge'] = df['Discharge']
    df['flow_direction'] = df.apply(lambda x: degrees(
        atan2(x[columns[3]], x[columns[2]])), axis=1)

    # Get the coordinates of the
    coord_list = list(sub_line.coords)
    start_coord = coord_list[0]
    end_coord = coord_list[1]
    angle_line = AngleBtw2Points(start_coord, end_coord)
    perpendicular_line = angle_line - 90

    # Update the columns, and calculate flow direction
    columns = df.columns
    df['Discharge [meter^3/s]'] = df.apply(lambda x: x[columns[4]] if calculate_flow_direction(
        perpendicular_line, x[columns[5]]) else x[columns[4]]*-1, axis=1)
    df['Discharge + [meter^3/s]'] = df.apply(
        lambda x: x[columns[4]] if x[columns[4]] > 0 else 0, axis=1)
    df['Discharge - [meter^3/s]'] = df.apply(
        lambda x: x[columns[4]] if x[columns[4]] < 0 else 0, axis=1)

    # Calculate the cumulative sum of the flows
    df['Cum. disc. + [meter^3]'] = np.cumsum(df['Discharge + [meter^3/s]'])
    df['Cum. disc. - [meter^3]'] = np.cumsum(df['Discharge - [meter^3/s]'])
    df['Cum. disc. [meter^3/s]'] = np.cumsum(df['Discharge [meter^3/s]'])
    return df[['time','Discharge + [meter^3/s]', 'Discharge - [meter^3/s]', 'Discharge [meter^3/s]', 'Cum. disc. + [meter^3]', 'Cum. disc. - [meter^3]', 'Cum. disc. [meter^3/s]']]


def get_values_for_linestring(grid_size, ds, geometry_row):

    # Number of points in the polyline
    num_coords = len(geometry_row.coords)

    # Convert gridsize to integer
    grid_size = int(grid_size)

    # We will take timesteps equal to grid_size/2 in order not to miss any of the cells.
    # However, in order to not duplicate any of the cells, we need to check whether the coordinates have been calculated before
    calculated_coords = []

    # Loop through each coordinate, create a new polyline for each and extract result
    for j in range(1, num_coords):

        # Define start and end coordinates
        starting_coord = geometry_row.coords[j-1]
        end_coord = geometry_row.coords[j]
        # Create new LineString of new coordinates
        sub_line = LineString([starting_coord, end_coord])

        # If this is the first coordinate in the linestring, create a new result_df that will later be concatenated with the other ones.
        val_dict = {}
        if j == 1:
            # Extract results
            point_array = ds.sel(x=starting_coord[0], y=starting_coord[1])
            time = point_array.time
            items = point_array.items
            val_dict['time'] = time

            # Extract results for each item in the dfs2
            for item in items:
                val_1 = point_array[item].values
                val_dict[str(item)] = val_1
            df_result = pd.DataFrame(index=time, data=val_dict)
            df_result = calculate_flow_magnitude(
                df_result, grid_size, sub_line)
            # Add the coordinates of point_array to our memory list
            calculated_coords.append(
                [point_array.geometry.x, point_array.geometry.y])

        # Round the length of the line to closest integer and convert
        length_of_line = int(
            np.round(Point(starting_coord).distance(Point(end_coord)), 0))

        # Loop thorugh the values in the dfs2 file, with the step size equivalent to the gridsize
        for i in range(0, length_of_line, grid_size):

            # Reset the values dictionary
            val_dict = {}

            # Get coordinates of point with i distance from start coordinate
            ip = sub_line.interpolate(distance=i)

            # Extract results from point
            point_array = ds.sel(x=ip.coords[0][0], y=ip.coords[0][1])

            # Check whether the point have been calculated before
            new_coord_list = [point_array.geometry.x, point_array.geometry.y]
            if new_coord_list not in calculated_coords:

                # Extract values and time
                time = point_array.time
                items = point_array.items
                val_dict['time'] = time

                # append the coordinates to calculated_coords
                calculated_coords.append(new_coord_list)

                # Extract results for each item in the dfs2
                for item in items:
                    val_1 = point_array[item].values
                    val_dict[str(item)] = val_1

                # Create a df with this cross sections values
                df_next_coord = pd.DataFrame(index=time, data=val_dict)

                # Calculate flow magnitude and its direction
                df_next_coord = calculate_flow_magnitude(
                    df_next_coord, grid_size, sub_line)
                # Concatenate the results of the new coordinate to result_df. Note that here they are concatenated vertically

                # The vertical concatenation will be later dissolved using the groupby argument (for the time column) and taken as mean

                df_result = pd.concat([df_result, df_next_coord])

    # TEST! Include last coord.
    # Extract results
    point_array = ds.sel(x=end_coord[0], y=end_coord[1])
    time = point_array.time
    items = point_array.items
    val_dict['time'] = time
    new_coord_list = [point_array.geometry.x, point_array.geometry.y]

    if new_coord_list not in calculated_coords:
        # Extract results for each item in the dfs2
        # print("Du missade sista!!!")
        for item in items:
            val_1 = point_array[item].values
            val_dict[str(item)] = val_1
        df_next = pd.DataFrame(index=time, data=val_dict)
        df_next_coord = calculate_flow_magnitude(
            df_next, grid_size, sub_line)
        # Add the coordinates of point_array to our memory list
        df_result = pd.concat([df_result, df_next_coord])
        calculated_coords.append(new_coord_list)

    df_result = df_result.groupby('time', as_index=False).sum()
    # For swedish users, the , is the default separator, instead of . (This is a feature requested by the user)
    # columns = df_result.columns
    # for col in columns:
    #    df_result[col] = df_result[col].astype(
    #        str).str.replace('.', ',', regex=False)

    return df_result


def dfs2tocsv(filepath_ds, filepath_shape,output_dir_and_name):

    # Check whether it is a dfs2 file or not
    if filepath_ds[-4:] != 'dfs2':
        print("Please pick a dfs2 file")
        return False

    # Read dfs2
    ds = mi.read(filepath_ds)

    # Check whether it is a shp file or not
    if filepath_shape[-3:] != 'shp':
        print("Please pick a shapefile")
        return False

    # Read shapefile
    shape_df = gdp.read_file(filepath_shape)

    # Check whether the file is a Polyline
    if not isinstance(shape_df['geometry'][0], LineString):
        print('The file is not a Polyline, please specify a Polyline instead.')
        return False
    
    index_check = 0
    #Check whether the ids are unique in the shapefile
    if len(shape_df['Id']) != shape_df['Id'].nunique():
        index_check = 1
        print('The values in the Id column in the shapefile is not unique. The sheets will be named after polyline index')

    # Get grid size
    grid_size = get_grid_size(ds)

    # Dictionary with results
    df_dict = {}

    # Loop through df with the shapefile and save the results
    for index, row in shape_df.iterrows():

        # Check direction. The calculations should always be done from left to right.
        first_coord = row['geometry'].coords[0]
        last_coord = row['geometry'].coords[len(row['geometry'].coords)-1]
        angle = AngleBtw2Points(first_coord, last_coord)

        # If angle is in the second or third quadrant, the general direction is right, so it must be reversed
        if (angle > 90) | (angle < -90):
            row['geometry'] = reverse_geom(row['geometry'])
            #print(
            #    'The cross section is defined from right to left. The cross section will be reversed.')
        if index_check == 1:
            df_dict[index] = get_values_for_linestring(
                grid_size, ds, row['geometry'])
        else:
            df_dict[row['Id']] = get_values_for_linestring(
                grid_size, ds, row['geometry'])
        

    # Create a multilevel indexed dataframe for easier read in the csv format (requested by users)
    resultant_df = pd.concat(df_dict.values(), axis=1, keys=df_dict.keys())
    #resultant_df.to_csv(output_dir_and_name, sep=';')

    #Try with excelwriter
    with pd.ExcelWriter(output_dir_and_name) as writer:
        for key, value in df_dict.items():
            value.to_excel(writer,sheet_name=str(key))
    #return resultant_df


# if __name__ == '__main__':
#    dfs2tocsv(sys.argv[1], sys.argv[2], sys.argv[3])


In [41]:
filepath_ds = r'C:\Project folder\python_specialuppdrag\MASV specialuppdrag\06_Linnestaden_HPQ.dfs2'
filepath_shp = r'C:\Project folder\python_specialuppdrag\MASV specialuppdrag\Flödessektioner_new_230206.shp'
output_dir = r'C:\Project folder\python_specialuppdrag\output_csvs\heja.xls'
#test = dfs2tocsv(filepath_ds,filepath_shp)
ds = mi.read(filepath_ds)
shape_df = gdp.read_file(filepath_shp)
grid_size= get_grid_size(ds)
#test,df_check= get_values_for_linestring(grid_size, ds, shape_df['geometry'][0])
dfs2tocsv(filepath_ds,filepath_shp,output_dir)

The values in the Id column in the shapefile is not unique. The sheets will be named after polyline index


C:\Users\masv\AppData\Local\Temp\ipykernel_10308\3546662076.py:254: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  with pd.ExcelWriter(output_dir_and_name) as writer:


0                                                   \
                   time Discharge + [meter^3/s] Discharge - [meter^3/s]   
0   2014-07-26 17:45:00            0.000000e+00                0.000000   
1   2014-07-26 17:50:00            0.000000e+00               -0.061580   
2   2014-07-26 17:55:00            0.000000e+00               -0.098016   
3   2014-07-26 18:00:00            0.000000e+00               -0.201111   
4   2014-07-26 18:05:00            1.409832e-10               -0.395818   
..                  ...                     ...                     ...   
143 2014-07-27 05:40:00            0.000000e+00               -0.000062   
144 2014-07-27 05:45:00            0.000000e+00               -0.000057   
145 2014-07-27 05:50:00            0.000000e+00               -0.000052   
146 2014-07-27 05:55:00            0.000000e+00               -0.000046   
147 2014-07-27 06:00:00            0.000000e+00               -0.000040   

                                                                         \
    Discharge [meter^3/s] Cum. disc. + [meter^3] Cum. disc. - [meter^3]   
0                0.000000           0.000000e+00               0.000000   
1               -0.061580           0.000000e+00              -0.061580   
2               -0.098016           0.000000e+00              -0.159596   
3               -0.201111           0.000000e+00              -0.360707   
4               -0.395818           1.409832e-10              -0.756525   
..                    ...                    ...                    ...   
143             -0.000062           1.793898e-04             -30.604105   
144             -0.000057           1.793898e-04             -30.604162   
145             -0.000052           1.793898e-04             -30.604214   
146             -0.000046           1.793898e-04             -30.604260   
147             -0.000040           1.793898e-04             -30.604300   

                                            1                           \
    Cum. disc. [meter^3/s]                time Discharge + [meter^3/s]   
0                 0.000000 2014-07-26 17:45:00            0.000000e+00   
1                -0.061580 2014-07-26 17:50:00            0.000000e+00   
2                -0.159596 2014-07-26 17:55:00            0.000000e+00   
3                -0.360707 2014-07-26 18:00:00            4.272476e-05   
4                -0.756525 2014-07-26 18:05:00            0.000000e+00   
..                     ...                 ...                     ...   
143             -30.603926 2014-07-27 05:40:00            3.567169e-20   
144             -30.603983 2014-07-27 05:45:00            3.567169e-20   
145             -30.604034 2014-07-27 05:50:00            3.567169e-20   
146             -30.604080 2014-07-27 05:55:00            3.567169e-20   
147             -30.604121 2014-07-27 06:00:00            3.418553e-20   

                             ...                     13  \
    Discharge - [meter^3/s]  ... Cum. disc. + [meter^3]   
0                  0.000000  ...                      0   
1                  0.000000  ...                      0   
2                 -0.000187  ...                      0   
3                 -0.000218  ...                      0   
4                 -0.000184  ...                      0   
..                      ...  ...                    ...   
143                0.000000  ...                      0   
144                0.000000  ...                      0   
145                0.000000  ...                      0   
146                0.000000  ...                      0   
147                0.000000  ...                      0   

                                                                   14  \
    Cum. disc. - [meter^3] Cum. disc. [meter^3/s]                time   
0             0.000000e+00           0.000000e+00 2014-07-26 17:45:00   
1             0.000000e+00           0.000000e+00 2014-07-26 17:50:00   
2             0.000000e+00           0.000000e

In [33]:
shape_df

,Id,geometry
0,0,"LINESTRING (147277.162 6395987.295, 147230.331..."
1,0,"LINESTRING (147110.474 6396180.177, 147070.787..."
2,0,"LINESTRING (147129.485 6396647.366, 147145.448..."
3,0,"LINESTRING (147136.584 6396343.359, 147085.385..."
4,0,"LINESTRING (146600.590 6395470.285, 146622.286..."
5,0,"LINESTRING (146741.878 6395646.498, 146768.337..."
6,0,"LINESTRING (146927.193 6395908.224, 146951.005..."
7,0,"LINESTRING (146695.363 6395766.627, 146748.809..."
8,0,"LINESTRING (146692.585 6395575.465, 146816.278..."
9,0,"LINESTRING (146973.043 6396021.289, 147022.785..."


In [9]:
df_check.to_csv('df_check.csv',sep=';')

In [47]:
to_shapefile = pd.DataFrame(data = {'coordinate_points':coords_list})
from shapely.geometry import Point
to_shapefile['coordinate_points'] = to_shapefile['coordinate_points'].apply(Point)
df = gdp.GeoDataFrame(to_shapefile,geometry='coordinate_points')
df.to_file(r'C:\Project folder\python_specialuppdrag\MASV specialuppdrag\endast_tvo_comp.shp',driver = 'ESRI Shapefile')

C:\Users\masv\AppData\Local\Temp\ipykernel_16000\1494209363.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file(r'C:\Project folder\python_specialuppdrag\MASV specialuppdrag\endast_tvo_comp.shp',driver = 'ESRI Shapefile')


,coordinate_points
0,POINT (147127.75 6396646.750004)
1,POINT (147131.75 6396642.750004)
2,POINT (147131.75 6396638.750004)
3,POINT (147135.75 6396634.750004)
4,POINT (147139.75 6396630.750004)
5,POINT (147139.75 6396626.750004)
6,POINT (147139.75 6396622.750004)
7,POINT (147143.75 6396618.750004)
8,POINT (147143.75 6396614.750004)
9,POINT (147147.75 6396610.750004)


In [70]:
shape_df = gdp.read_file(filepath_shp)
shape_df.geometry[0].coords[4]

IndexError: index out of range

In [9]:
test['Resultant_flow'] = test.apply(lambda x: hypot(x['P flux <Flow Flux> (meter pow 3 per sec per meter)'],x['Q flux <Flow Flux> (meter pow 3 per sec per meter)']),axis=1)
test['degrees'] = test.apply(lambda x:degrees(atan2(x['P flux <Flow Flux> (meter pow 3 per sec per meter)'],x['Q flux <Flow Flux> (meter pow 3 per sec per meter)'])),axis=1)

In [21]:
test.columns[2]

'P flux <Flow Flux> (meter pow 3 per sec per meter)'

#### NOTE TO AFTERNOON! if angle == (perpendicular+- 180): positive ,else negative (sort of) 


- No we have to extract at which SIDE the extraction point is from and then done. 

In [11]:
test2=ds.sel(x = lineList[0][0],y=lineList[0][1])

In [19]:
50/4

12.5

In [13]:
test2.geometry.x

147275.75

In [14]:
extracted_point_x = test2.geometry.x
extracted_point_y = test2.geometry.y

In [15]:
extracted_point_y

6395986.750004

In [16]:
atan2(x,y)

-0.38227428775020417

In [17]:
angle_line

-158.7026459548892

In [18]:
import shapely.wkt
import shapely.ops


def reverse_geom(geom):
    def _reverse(x, y, z=None):
        if z:
            return x[::-1], y[::-1], z[::-1]
        return x[::-1], y[::-1]

    return shapely.ops.transform(_reverse, geom)

test3 = reverse_geom(line)

In [19]:
test3.wkt

'LINESTRING (147230.33088362962 6395969.038813498, 147277.16222729348 6395987.295100007)'

In [20]:
line.wkt

'LINESTRING (147277.16222729348 6395987.295100007, 147230.33088362962 6395969.038813498)'

In [37]:
-7.829035669739706/-8.36763

0.935633586779017